# convertParquetMPI

This notebook reads in patient data from an uploaded parquet file (`mpi_incoming_file_path`), converts the data to FHIR bundles, and posts the FHIR bundles to the Record Linkage endpoint.

In [ ]:
pip install azure-identity phdi

Set up parameters for connecting to the storage account, vault client, and record linkage container app.

In [ ]:
from pyspark.sql import SparkSession
from notebookutils import mssparkutils
from phdi.linkage.seed import convert_to_patient_fhir_resources
from azure.identity import DefaultAzureCredential
import requests
import time
from datetime import date
from phdi.transport.http import http_request_with_retry
from phdi.fhir.harmonization.standardization import (
    standardize_names,
    standardize_phones,
    standardize_dob,
)
# from phdi.cloud.azure import AzureCloudContainerConnection, AzureCredentialManager
from phdi.fhir.geospatial import SmartyFhirGeocodeClient
import json

spark = SparkSession.builder.getOrCreate()

# Set up file client and endpoint credentials
account_name = "$STORAGE_ACCOUNT"
endpoint_scope = "$SCOPE"
snapshot_bucket_name = "bundle-snapshots"
file_system_name = "patient-data"
file_path = "MPI.parquet"
storage_account_url = f"https://{account_name}.blob.core.windows.net/"
mpi_incoming_file_path = f"abfss://{file_system_name}@{account_name}.dfs.core.windows.net/{file_path}"

# Set up for writing to blob storage
blob_relative_path = ""
linked_service_name = "AzureBlobStorage1" 
blob_sas_token = mssparkutils.credentials.getConnectionStringOrCreds(linked_service_name)
wasb_path = 'wasbs://%s@%s.blob.core.windows.net/%s' % (snapshot_bucket_name, account_name, blob_relative_path)
spark.conf.set('fs.azure.sas.%s.%s.blob.core.windows.net' % (snapshot_bucket_name, account_name), blob_sas_token)
# Try mounting the remote storage directory at the mount point
try:
    mssparkutils.fs.mount(
        wasb_path,
        "/",
        {"LinkedService": "AzureBlobStorage1"}
    )
except:
    print("Already mounted")

# Set up key vault client
vault_name = "$KEY_VAULT"
vault_url = f"https://{vault_name}.vault.azure.net/"
vault_linked_service = "$KEY_VAULT_LINKED_SERVICE"

credential = DefaultAzureCredential()

# Get client ID and secret for GitHub app registration
client_id = TokenLibrary.getSecret(vault_name,"synapse-client-id",vault_linked_service)
client_secret = TokenLibrary.getSecret(vault_name,"synapse-client-secret",vault_linked_service)


# Get smarty client information
smarty_auth_id = TokenLibrary.getSecret(vault_name,"smarty-auth-id",vault_linked_service)
smarty_auth_token = TokenLibrary.getSecret(vault_name,"smarty-auth-token",vault_linked_service)

# Get access token for record linkage container app
tenant_id = "$TENANT_ID"
endpoint_login_url = f"https://login.microsoftonline.com/{tenant_id}/oauth2/v2.0/token"

# # # Set up Azure cloud managers
# azure_credential_manager = AzureCredentialManager(resource_location=storage_account_url, 
#                                             credential_type="service-principle", 
#                                             client_id=client_id, 
#                                             tenant_id=tenant_id, 
#                                             client_secret=client_secret)
# azure_connection_manager = AzureCloudContainerConnection(cred_manager=azure_credential_manager,storage_account_url=storage_account_url)


# Set up for record linkage and ingestion access
def set_up_endpoint_access(endpoint, url):
    data = {
        'grant_type': 'client_credentials',
        'client_id': client_id,
        'client_secret': client_secret,
        'scope': f'api://{endpoint_scope}-{endpoint}/.default'
    }

    response = requests.post(url, data=data)
    access_token = response.json()['access_token']
    headers = {
        'Authorization': f'Bearer {access_token}'
    }
    return data, headers

record_linkage_data, record_linkage_headers = set_up_endpoint_access(endpoint="record-linkage",url=endpoint_login_url)
ingestion_data, ingestion_headers = set_up_endpoint_access(endpoint="ingestion",url=endpoint_login_url)

# Define function for refreshing Bearer access token
def request_with_token_refresh(url,headers,data,endpoint_type,endpoint_login_url,try_again=True):
    resp = http_request_with_retry(url=url,retry_count=2,request_type="POST",headers=headers,allowed_methods=["POST"],data=data)
    if resp.status_code == 401 and try_again:
        _, headers = set_up_endpoint_access(endpoint=endpoint_type,url=endpoint_login_url)
        print("got new headers")
        return request_with_token_refresh(url,headers,data,endpoint_type,endpoint_login_url,try_again=False)
    if resp.status_code != 200:
        print(resp.text)
    try:
        resp.raise_for_status()
    except Exception as e:
        print("request failed")
        print(e)
        print(url)
        print(data)
        print(headers)
        print(endpoint_type)
        print()

    return resp

# Get URLs for container apps
record_linkage_url = TokenLibrary.getSecret(vault_name,"record-linkage-url",vault_linked_service)+ "/link-record" 
ingestion_url = TokenLibrary.getSecret(vault_name,"ingestion-url",vault_linked_service)
write_to_storage_url = ingestion_url + "/cloud/storage/write_blob_to_storage"

# Instantiate Smarty Client
geocode_client = SmartyFhirGeocodeClient(
                smarty_auth_id=smarty_auth_id,
                smarty_auth_token=smarty_auth_token,
                licenses=["us-rooftop-geocoding-enterprise-cloud"]
            )

Read the MPI parquet data into a spark dataframe. Iterate over each row of patient data in the dataframe and convert to a FHIR bundle and associated iris_id. Create a POST request to the record linkage container with FHIR bundle and iris_id.

In [ ]:
# Convert and post data from mpi_incoming_file_path
def convert(mpi_incoming_file_path):
    df = spark.read.parquet(mpi_incoming_file_path)
    curr_date = date.today()
    file_idx = 0
    for row in df.collect()[0:4]:
        file_idx += 1
        print(file_idx)
        iris_id, fhir_bundle = convert_to_patient_fhir_resources(row.asDict())

        # Add metadata to LAC Extract patientdenoting it came from IRIS
        # We know this URI is ~probably~ not right but Brady might know what to use...
        patients = [
            r for r in fhir_bundle.get("entry", []) if r.get("resource", {}).get("resourceType", "") == "Patient"
        ]
        patients[0]["meta"] = {"source": "uri:iris"}
        data = {
            'bundle': fhir_bundle,
            'external_person_id': iris_id
        }


        # Store a snapshot of the FHIR bundle before it hits standardization
        pre_filename = f"abfss://{snapshot_bucket_name}@{account_name}.dfs.core.windows.net/pre/lac_extract_{str(curr_date)}_{str(file_idx)}.json"
        pre_write_data = {
            'blob': data["bundle"],
            'cloud_provider': 'azure',
            'bucket_name': snapshot_bucket_name,
            'file_name': pre_filename,
            'storage_account_url':storage_account_url

        }
        mssparkutils.fs.put(pre_filename, json.dumps(data['bundle']), True)
        # azure_connection_manager.upload_object(message=data["bundle"],container_name=snapshot_bucket_name,filename=pre_filename)
        # print(file_idx, "pre_write_to_storage")
        

        # Perform pipeline standardization on extracted data to mirror how non-extracted data gets handled by the pipeline
        data['bundle'] = standardize_names(data = data["bundle"])
        print(file_idx, "std_names")

        data['bundle'] = standardize_phones(data = data["bundle"])
        print(file_idx, "std_phones")

        try:
            data['bundle'] = standardize_dob(data = data['bundle'])
            print(file_idx, "std_dob")
        except Exception as e:
            continue

        data['bundle'] = geocode_client.geocode_bundle(bundle = data['bundle'])
        print(file_idx, "std_address")

        # Also store a copy of the bundle after it hits standardization
        # post_file_name = f"/post/lac_extract_{str(curr_date)}_{str(file_idx)}.json"
        post_filename = f"abfss://{snapshot_bucket_name}@{account_name}.dfs.core.windows.net/post/lac_extract_{str(curr_date)}_{str(file_idx)}.json"

        post_write_data = {
            'blob': data["bundle"],
            'cloud_provider': 'azure',
            'bucket_name': snapshot_bucket_name,
            'file_name': post_filename,
            'storage_account_url':storage_account_url
        } 
        mssparkutils.fs.put(post_filename, json.dumps(data['bundle']), True)
        # azure_connection_manager.upload_object(message=data["bundle"],container_name=snapshot_bucket_name,filename=post_file_name)
        # print(file_idx, "post_write_to_storage")
     
        # Now we can send the extracted record off to the MPI
        resp = request_with_token_refresh(url=record_linkage_url,headers=record_linkage_headers,data=data,endpoint_type="record-linkage",endpoint_login_url=endpoint_login_url,try_again=True)
        print(file_idx, "record_linkage", resp.status_code)
        if resp.status_code >= 400:
            continue
        

convert(mpi_incoming_file_path)